In [105]:
import os
import github as gh
import bs4 as bs
import requests
from lxml import etree

import selenium
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm

import re

In [106]:
GITHUB_ACCESS_TOKEN=os.getenv('GITHUB_ACCESS_TOKEN')
g=gh.Github(GITHUB_ACCESS_TOKEN)
print(g.get_user())

AuthenticatedUser(login=None)


In [107]:
query='tensorflow language:python created:2021-04-01..2021-04-02'
urls = []
result=g.search_repositories(query)
for repository in result:
    urls.append(repository.html_url)
# print(result.totalCount)
# print(dir(result))
    

In [108]:
url='https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras'
url2='https://github.com/bamblebam/image-classification-rps'

In [109]:
options=Options()
options.headless=True
driver=webdriver.Chrome(ChromeDriverManager(path='./').install(),options=options)
driver.get(url2)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [./\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [110]:
table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")

In [111]:
# links=table.find_elements_by_tag_name('a')
# print(len(links))
# for link in links:
#     href=link.get_attribute('href')
#     if 'tree' in href or '.py' in href:
#         stack.append(href)

# driver.get(url2)
# table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
# links=table.find_elements_by_tag_name('a')
# for link in links:
#     href=link.get_attribute('href')
#     if '/tree/' in href or '.py' in href:
#         stack.append(href)

In [125]:
queue=list()
full_list=list()
links_list = set()
sequential_list=list()

In [113]:
def push_to_queue(links):
    for link in links:
        href=link.get_attribute('href')
        if href in links_list:
            continue
        if '/tree/' in href or '.py' in href:
            links_list.add(href)
            queue.append(href)

In [114]:
# def if_sequential():
#     code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
#     if 'Sequential' in code_body.text:
#             count+=1
#     return count

In [115]:
def search_through_files(link):
    print(link)
    if '/tree/' in link and not 'venv' in link:
        driver.get(link)
        time.sleep(2.5)
        try:
            table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-block']")
            links=table.find_elements_by_tag_name('a')
            push_to_queue(links)
        except:
            print("No element found")
    elif '.py' in link and not 'venv' in link:
        driver.get(link)
        time.sleep(2.5)
        full_list.append(link)
        try: 
            code_body=driver.find_element_by_xpath("//*[@class='highlight tab-size js-file-line-container']")
            if 'Sequential' in code_body.text:
                sequential_list.append(url)
        except:
            print('Wrong class selector') 

In [116]:
def bfs():
    while queue:
        link=queue.pop(0)
        search_through_files(link)

In [117]:
def get_all_relevant_links(url):
    links_list.add(url)
    driver.get(url)
    try:
        table=driver.find_element_by_xpath("//*[@class='Details-content--hidden-not-important js-navigation-container js-active-navigation-container d-md-block']")
        links=table.find_elements_by_tag_name('a')
        for link in links:
            href=link.get_attribute('href')
            if ('/tree/' in href or '.py' in href) and not 'venv' in href:
                links_list.add(href)
                queue.append(href)
    except:
        print('No element found')
    bfs()
    print("Ended")

In [141]:

def getLayerSequence2(code):
    '''
    This is the function for models of type model.add(layerName)
    :param code: the code to check layer sequence
    :return: array in correct sequence of layers
    :rtype: array 
    '''
    rawLayerSequence = re.findall('model.add(.*)',code)
    layerSequence = []
    for i in rawLayerSequence:
        temp = re.findall('keras.layers.?(.*)\(',i)
        layerSequence.append(temp[0])
    return layerSequence
    


In [142]:
def getSequentialModel(link):
    layerSequence = []
    get_all_relevant_links(link)
    for url in full_list:
        driver.get(url)
        time.sleep(2.5)
        print(url)
        try:
            code_body=driver.find_element_by_xpath("//*[@class='Box-body p-0 blob-wrapper data type-python  gist-border-0']")
            if 'Sequential' in code_body.text:
                layers = getLayerSequence2(code_body.text)
                print(layers, '\n')
                layerSequence.append(layers)
        except e:
            print(e)
    return layerSequence

In [127]:
testUrl = 'https://github.com/fadhil-code/TensorFlowProject3'
count = getSequentialModel(testUrl)

[['(keras.layers.Dense(N_HIDDEN,', '(keras.layers.Dropout(DROPOUT))', '(keras.layers.Dense(N_HIDDEN,', '(keras.layers.Dropout(DROPOUT))', '(keras.layers.Dense(NB_CLASSES,'], ['(keras.layers.Dense(N_HIDDEN,', '(keras.layers.Dense(N_HIDDEN,', '(keras.layers.Dense(NB_CLASSES,'], ['(keras.layers.Dense(NB_CLASSES,']]
['https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras', 'https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras', 'https://github.com/Tyler-Shamsuddoha/python-image-classifier-keras']


In [143]:
print(count)
print(sequential_list)

https://github.com/fadhil-code/TensorFlowProject3/blob/master/Dropout.py
https://github.com/fadhil-code/TensorFlowProject3/blob/master/hidden_layers.py
https://github.com/fadhil-code/TensorFlowProject3/blob/master/main.py
Ended
https://github.com/fadhil-code/TensorFlowProject3/blob/master/Dropout.py
['Dense', 'Dropout', 'Dense', 'Dropout', 'Dense'] 

https://github.com/fadhil-code/TensorFlowProject3/blob/master/hidden_layers.py
['Dense', 'Dense', 'Dense'] 

https://github.com/fadhil-code/TensorFlowProject3/blob/master/main.py
['Dense'] 

https://github.com/fadhil-code/TensorFlowProject3/blob/master/Dropout.py
['Dense', 'Dropout', 'Dense', 'Dropout', 'Dense'] 

https://github.com/fadhil-code/TensorFlowProject3/blob/master/hidden_layers.py
['Dense', 'Dense', 'Dense'] 

https://github.com/fadhil-code/TensorFlowProject3/blob/master/main.py
['Dense'] 



In [144]:
for i in count:
    print(i)

['Dense', 'Dropout', 'Dense', 'Dropout', 'Dense']
['Dense', 'Dense', 'Dense']
['Dense']
['Dense', 'Dropout', 'Dense', 'Dropout', 'Dense']
['Dense', 'Dense', 'Dense']
['Dense']
